In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.preprocessing import image
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential

In [ ]:
train_datagenerator = image.ImageDataGenerator(
    rescale = 1/255, horizontal_flip = True, zoom_range = 0.2, shear_range = 0.2
)

train_data = train_datagenerator.flow_from_directory(directory="../input/covid-data-nmtraining/kaggle_covid_data_test/train",
                                                     target_size = (256, 256), 
                                                     batch_size = 16, 
                                                     class_mode = "binary")

In [ ]:
train_data.class_indices

In [ ]:
test_datagenerator = image.ImageDataGenerator(
    rescale = 1/255
)

test_data = test_datagenerator.flow_from_directory(directory="../input/covid-data-nmtraining/kaggle_covid_data_test/test",
                                                     target_size = (256, 256), 
                                                     batch_size = 16, 
                                                     class_mode = "binary")

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(Conv2D(filters = 64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
#model.add(Dropout(rate=0.25))

model.add(Conv2D(filters = 64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
#model.add(Dropout(rate=0.25))

model.add(Conv2D(filters = 128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
#model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=96, activation='relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])
#model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(train_data, epochs=10,
                    validation_data = test_data)

In [ ]:
import os
from keras.preprocessing import image
CATEGORIES = ['Covid19', 'Normal']
#path = "../input/customtest/val/Covid19/COVID-3352.png";
def conv_image(path):
    img = image.load_img(path, target_size=(256, 256))
    img = image.img_to_array(img)/255
    img = np.array([img])
    return img

In [ ]:
DIR = r'../input/customtest/val';
label = 0
for category in CATEGORIES:
    path = os.path.join(DIR, category)
    #print(path)
    count = 0;
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        prediction = model.predict(conv_image(img_path))
        #print(CATEGORIES[prediction.argmax()])
        answer = round(prediction[0][0])
        if(CATEGORIES[answer] == category):
            count = count + 1;
    if(label == 0):
      print(f"{category} prediction : {count/156}");
      #break;
    else:
      print(f"{category} prediction : {count/200}");
    label = label+1;
#prediction = model.predict(img)

In [ ]:
#model.save("model/covid_new_.h5")